### CORONA VIRUS BASIC VOICE ASSISTANT TO FIND TOTAL CASES OR DEATHS

In [3]:
API_KEY="tWJqQ22uVMQk"
RUN_TOKEN="t4DjxckgHtnu"
PROJECT_TOKEN="tik6V2ieTsfu"

In [4]:
import requests
import json
import time
import threading
import pyttsx3
import speech_recognition as s
import re

In [5]:
response=requests.get(f'https://www.parsehub.com/api/v2/projects/{PROJECT_TOKEN}/last_ready_run/data',params={"api_key":API_KEY})

In [6]:
data=json.loads(response.text)
#print(data)

In [13]:
class data:
    def __init__(self,api_key,project_token):
        self.api_key=api_key
        self.project_token=project_token
        self.params={"api_key":api_key}
        self.data=self.getupdate()
    
    def getcountrylist(self):
        countrylist=[]
        for country in self.data["countries"]:
            countrylist.append(country['name'].lower())
            
        return countrylist    
        
        
    def getupdate(self):
        response=requests.get(f'https://www.parsehub.com/api/v2/projects/{PROJECT_TOKEN}/last_ready_run/data',params={"api_key":API_KEY})
        data=json.loads(response.text)
        return data
    
    def gettotalcases(self):
        data= self.data['total']
        for c in data:
            if c['name']=="Coronavirus Cases:":
                return c['value']
            
        
    def gettotaldeaths(self):
        data= self.data['total']
        for c in data:
            if c['name']=="Deaths:":
                return c['value']   
            
            
            
    def getcountry(self,countries):
        data= self.data["countries"]
        for c in data:
            if c['name'].lower()==countries.lower():
                return c
    
    def updateinfo(self):
        response=requests.post(f'https://www.parsehub.com/api/v2/projects/{PROJECT_TOKEN}/run',params={"api_key":API_KEY})
    
        
        def poll():
            time.sleep(0.1)
            olddata=self.data
            while True:
                newdata=self.getupdate()
                if newdata!=olddata:
                    self.data=newdata
                    print("data updated")
                    break
                time.sleep(5)   
                
        t=threading.Thread(target=poll)
        t.start()
        
        
        
            
            

In [14]:
data=data(API_KEY,PROJECT_TOKEN)
print(data.getcountry("india"))
print(data.getcountrylist())

{'name': 'India', 'total_cases': '5,112,431', 'new_cases': '+94,397', 'total_deaths': '83,208'}
['usa', 'india', 'brazil', 'russia', 'peru', 'colombia', 'mexico', 'south africa', 'spain', 'argentina', 'chile', 'iran', 'france', 'uk', 'bangladesh', 'saudi arabia', 'pakistan', 'iraq', 'turkey', 'italy', 'philippines', 'germany', 'indonesia', 'israel', 'ukraine', 'canada', 'bolivia', 'qatar', 'ecuador', 'romania', 'kazakhstan', 'dominican republic', 'panama', 'egypt', 'kuwait', 'belgium', 'morocco', 'oman', 'sweden', 'netherlands', 'china', 'guatemala', 'uae', 'japan', 'poland', 'belarus', 'honduras', 'ethiopia', 'portugal', 'venezuela', 'bahrain', 'nepal', 'costa rica', 'singapore', 'nigeria', 'uzbekistan', 'algeria', 'switzerland', 'armenia', 'ghana', 'kyrgyzstan', 'moldova', 'czechia', 'afghanistan', 'azerbaijan', 'kenya', 'austria', 'palestine', 'serbia', 'ireland', 'paraguay', 'el salvador', 'australia', 'lebanon', 'libya', 'bosnia and herzegovina', 's. korea', 'denmark', 'cameroon',

In [15]:
def speak(text):
    engine=pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def getaudio():
    r=s.Recognizer()
    with s.Microphone() as source:
        audio=r.listen(source)
        said=""
        try:
            said=r.recognize_google(audio)
        except Exception as e:
            print("Exception:",str(e))
    return said.lower()


In [16]:
#speak("hello")

In [17]:
def main():
    countrylist=list(data.getcountrylist());
    TOTALPATTERNS={
        re.compile("[\w\s]+total [\w\s]+ cases"):data.gettotalcases,
        re.compile("[\w\s]+total [\w\s]+ deaths"):data.gettotaldeaths,
        re.compile("[\w\s]+total cases"):data.gettotalcases,
        re.compile("[\w\s]+total deaths"):data.gettotaldeaths
        
    }
    COUNTRYPATTERNS={
        re.compile("[\w\s]+ cases [\w\s]+"): lambda c: data.getcountry(c)['total_cases'],
         re.compile("[\w\s]+ death [\w\s]+"): lambda c: data.getcountry(c)['total_deaths'],
        re.compile("[\w\s]+ deaths [\w\s]+"): lambda c: data.getcountry(c)['total_deaths'],
        re.compile("death [\w\s]+"): lambda c: data.getcountry(c)['total_deaths'],
        re.compile("[\w\s]+ death[\w\s]+"): lambda c: data.getcountry(c)['total_deaths'],
        
    }
    
    UPDATECURRENT="update"
    
    print("START")
    
    END_PHRASE="stop"
    while True:
        print("Speak..")
        result=None
        text=getaudio()
        print(text)
        for p,func in COUNTRYPATTERNS.items():
            if p.match(text):
                words=set(text.split(" "))
                for c in countrylist:
                    if c in words:
                        result=func(c)
                break                    
        for p,func in TOTALPATTERNS.items():
            if p.match(text):
                result=func()
                break
        if text==UPDATECURRENT:
            result="data is being updated. this may take a moment! Please Wait!"
            data.updateinfo()       
        if result:
            speak(result)
        if text.find(END_PHRASE)!=-1:
            speak("exiting now. stay safe! Maintain Social distancing.")
            break
        
        

In [18]:
main()

START
Speak..
update
Speak..
stop
